In [54]:
from etherscan import Etherscan
from web3 import Web3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tsfresh import extract_features
from datetime import datetime
from os import listdir
from os.path import isfile, join
eth = Etherscan('FIT71X5XKTC5ZWC3UEQQ1BY1YV8TWXTDHM')

# Helper Functions

In [2]:
def get_address_stats_normal_tnx(data,address):
    sample_df = pd.DataFrame(data)
    # Column creation of ETH from Wei
    sample_df['eth value'] = sample_df['value'].apply(lambda x: Web3.fromWei(int(x),'ether'))

    # Typing of sent and received transactions
    sample_df['txn type'] = sample_df['from'].apply(lambda x: 'sent' if x== address else 'received')

    # Handling of Sent transactions stats
    sample_df_sent =  sample_df[sample_df['txn type'] == 'sent']
    sample_df_sent = sample_df_sent.sort_values(by=['timeStamp'])
    sample_df_sent['timeStamp'] = sample_df_sent['timeStamp'].astype('int')

    # Filtering of sent normal transfers to contract addresses
    sample_df_sent_contracts = sample_df[sample_df['contractAddress'] != '']

    # Compilation of normal sent transaction statistics
    core_stats_Sent_tnx = len(sample_df_sent)
    core_stats_MinValSent = sample_df_sent['eth value'].min()
    core_stats_MaxValSent = sample_df_sent['eth value'].max()
    core_stats_AvgValSent = sample_df_sent['eth value'].mean()
    core_stats_MinValueSentToContract = sample_df_sent_contracts['eth value'].min()
    core_stats_MaxValueSentToContract = sample_df_sent_contracts['eth value'].max()
    core_stats_AvgValueSentToContract = sample_df_sent_contracts['eth value'].mean()
    core_stats_TotalEtherSent = sample_df_sent['eth value'].sum()
    core_stats_TotalEtherSent_Contracts = sample_df_sent_contracts['eth value'].sum()
    core_stats_UniqueSentTo_Addresses = len(sample_df_sent['to'].unique())

    # Handling of received transactions stats
    sample_df_received =  sample_df[sample_df['txn type'] == 'received']
    sample_df_received = sample_df_received.sort_values(by=['timeStamp'])
    sample_df_received['timeStamp'] = sample_df_received['timeStamp'].astype('int')


    # Compilation of normal received transaction statistics
    core_stats_Received_tnx = len(sample_df_received)
    core_stats_MinValueReceived = sample_df_received['eth value'].min()
    core_stats_MaxValueReceived = sample_df_received['eth value'].max()
    core_stats_AvgValueReceived = sample_df_received['eth value'].mean()
    core_stats_TotalEtherReceived = sample_df_received['eth value'].sum()
    core_stats_UniqueReceivedFrom_Addresses = len(sample_df_received['from'].unique())

    # Handling of remaining normal transaction values
    sample_df['timeStamp'] = sample_df['timeStamp'].astype('int')
    sample_df.sort_values(by=['timeStamp'])
    sample_df['unix time difference'] = sample_df['timeStamp'].diff()
    sample_df_time_dim = sample_df.groupby('txn type')['unix time difference'].sum()/60


    # Compilation of remaining normal transaction statistics
    core_stats_TimeDiffbetweenfirstand_last = ((sample_df['timeStamp'].max()) - (sample_df['timeStamp'].min())) / 60
    core_stats_TotalTransactions = len(sample_df)
    core_stats_NumberofCreated_Contracts = len(sample_df[sample_df['contractAddress'] != ''])
    core_stats_Avg_min_between_received_tnx = sample_df_time_dim['received']/ core_stats_Received_tnx
    core_stats_Avg_min_between_sent_tnx = sample_df_time_dim['sent']/core_stats_Sent_tnx
    core_stats_TotalEtherBalance = core_stats_TotalEtherReceived - core_stats_TotalEtherSent
    compiled_normal_tnx_result = {'Address': address, 'FLAG': 1,
                                  'Avg min between sent tnx': core_stats_Avg_min_between_sent_tnx,
                                 'Avg min between received tnx': core_stats_Avg_min_between_received_tnx,
                                  'Time Diff between first and last (Mins)': core_stats_TimeDiffbetweenfirstand_last,
                                  'Sent tnx': core_stats_Sent_tnx, 'Received Tnx': core_stats_Received_tnx,
                                  'Number of Created Contracts': core_stats_NumberofCreated_Contracts,
                                  'Unique Received From Addresses':core_stats_UniqueReceivedFrom_Addresses,
                                  'Unique Sent To Addresses': core_stats_UniqueSentTo_Addresses,
                                  'min value received': core_stats_MinValueReceived,
                                  'max value received ': core_stats_MaxValueReceived,
                                  'avg val received': core_stats_AvgValueReceived,
                                  'min val sent': core_stats_MinValSent,
                                  'max val sent': core_stats_MaxValSent,
                                  'avg val sent': core_stats_AvgValSent ,
                                  'min value sent to contract': core_stats_MinValueSentToContract,
                                  'max val sent to contract': core_stats_MaxValueSentToContract,
                                  'avg value sent to contract': core_stats_AvgValueSentToContract,
                                  'total transactions (including tnx to create contract': core_stats_TotalTransactions,
                                  'total Ether sent': core_stats_TotalEtherSent,
                                  'total ether received': core_stats_TotalEtherReceived,
                                  'total ether sent contracts': core_stats_TotalEtherSent_Contracts,
                                  'total ether balance':core_stats_TotalEtherBalance}
    return compiled_normal_tnx_result

In [3]:
def get_address_normal_tnx(data,address):
    sample_df = pd.DataFrame(data)
    # Column creation of ETH from Wei
    sample_df['eth value'] = sample_df['value'].apply(lambda x: Web3.fromWei(int(x),'ether'))

    # Typing of sent and received transactions
    sample_df['txn type'] = sample_df['from'].apply(lambda x: 'sent' if x== address else 'received')
    sample_df['AccountId'] = address
    return sample_df

In [4]:
def get_address_stats_erc20_tnx(data,address):
    sample_df = pd.DataFrame(data)

    # Column creation of ETH from Wei
    sample_df['eth value'] = sample_df['value'].apply(lambda x: Web3.fromWei(int(x),'ether'))

    # Typing of sent and received ERC20 transactions
    sample_df['txn type'] = sample_df['from'].apply(lambda x: 'sent' if x == address else 'received')

    # Handling of Sent ERC20 transactions stats
    sample_df_sent =  sample_df[sample_df['txn type'] == 'sent']
    sample_df_sent = sample_df_sent.sort_values(by=['timeStamp'])
    sample_df_sent['timeStamp'] = sample_df_sent['timeStamp'].astype('int')

    # Creation of unix time difference column for Sent ERC20 transactions
    sample_df_sent['unix time diff'] = sample_df_sent['timeStamp'].diff()

    # Compilation of ERC20 sent transaction statistics
    core_stats_ERC20TotalEther_Sent = sample_df_sent['eth value'].sum()
    core_stats_ERC20TotalEtherSentContract = len(sample_df_sent[sample_df_sent['to'] != sample_df_sent['contractAddress']])
    core_stats_ERC20UniqSent_Addr = len(sample_df_sent['to'].unique())
    core_stats_ERC20AvgTimeBetweenSent_Tnx = ((sample_df_sent['unix time diff'].sum() / (len(sample_df_sent) - 1)) / 60) if (len(sample_df_sent) - 1) > 0 else 0
    core_stats_ERC20AvgTimeBetweenContract_Tnx = 0
    core_stats_ERC20MinVal_Sent = sample_df_sent['eth value'].min()
    core_stats_ERC20MaxVal_Sent = sample_df_sent['eth value'].max()
    core_stats_ERC20AvgVal_Sent = sample_df_sent['eth value'].mean()
    core_stats_ERC20UniqSentTokenName = len(sample_df_sent['tokenName'].unique())
    core_stats_ERC20MostSentTokenType = [sample_df_sent['tokenName'].mode().tolist()]

    # Handling of received ERC20 transactions stats
    sample_df_received =  sample_df[sample_df['txn type'] == 'received']
    sample_df_received = sample_df_received.sort_values(by=['timeStamp'])
    sample_df_received['timeStamp'] = sample_df_received['timeStamp'].astype('int')

    # Creation of unix time difference column for received ERC20 transactions
    sample_df_received['unix time diff'] = sample_df_received['timeStamp'].diff()

    # Compilation of ERC20 received transaction statistics
    core_stats_ERC20TotalEther_Received = sample_df_received['eth value'].sum()
    core_stats_ERC20UniqRec_Addr = len(sample_df_received['from'].unique())
    core_stats_ERC20UniqRecContractAddr = len(sample_df_received['contractAddress'].unique())
    core_stats_ERC20AvgTimeBetweenRec_Tnx = ((sample_df_received['unix time diff'].sum() / (len(sample_df_received) - 1)) / 60) if (len(sample_df_received) - 1) > 0 else 0
    core_stats_ERC20MinVal_Rec = sample_df_received['eth value'].min()
    core_stats_ERC20MaxVal_Rec = sample_df_received['eth value'].max()
    core_stats_ERC20AvgVal_Rec = sample_df_received['eth value'].mean()
    core_stats_ERC20UniqRecTokenName = len(sample_df_received['tokenName'].unique())
    core_stats_ERC20MostRecTokenType = [sample_df_received['tokenName'].mode().tolist()]

    # Compilation of remaining ERC20 transaction statistics
    core_stats_TotalERC20Tnxs = len(sample_df)
    compiled_ERC20_tnx_result = {'Address': address,
                                 ' Total ERC20 tnxs': core_stats_TotalERC20Tnxs,
                                 ' ERC20 total Ether received': core_stats_ERC20TotalEther_Received,
                                 ' ERC20 total ether sent': core_stats_ERC20TotalEther_Sent,
                                 ' ERC20 total Ether sent contract': core_stats_ERC20TotalEtherSentContract,
                                 ' ERC20 uniq sent addr': core_stats_ERC20UniqSent_Addr,
                                 ' ERC20 uniq rec addr': core_stats_ERC20UniqRec_Addr,
                                 ' ERC20 uniq sent addr.1': 0 ,
                                 ' ERC20 uniq rec contract addr': core_stats_ERC20UniqRecContractAddr,
                                 ' ERC20 avg time between sent tnx': core_stats_ERC20AvgTimeBetweenSent_Tnx,
                                 ' ERC20 avg time between rec tnx': core_stats_ERC20AvgTimeBetweenRec_Tnx,
                                 ' ERC20 avg time between rec 2 tnx': 0,
                                 ' ERC20 avg time between contract tnx': core_stats_ERC20AvgTimeBetweenContract_Tnx,
                                 ' ERC20 min val rec': core_stats_ERC20MinVal_Rec,
                                 ' ERC20 max val rec': core_stats_ERC20MaxVal_Rec ,
                                 ' ERC20 avg val rec': core_stats_ERC20AvgVal_Rec,
                                 ' ERC20 min val sent': core_stats_ERC20MinVal_Sent,
                                 ' ERC20 max val sent': core_stats_ERC20MaxVal_Sent,
                                 ' ERC20 avg val sent': core_stats_ERC20AvgVal_Sent,
                                 ' ERC20 min val sent contract': 0,
                                 ' ERC20 max val sent contract': 0,
                                 ' ERC20 avg val sent contract': 0,
                                 ' ERC20 uniq sent token name': core_stats_ERC20UniqSentTokenName,
                                 ' ERC20 uniq rec token name': core_stats_ERC20UniqRecTokenName,
                                 ' ERC20 most sent token type': core_stats_ERC20MostSentTokenType,
                                 ' ERC20_most_rec_token_type': core_stats_ERC20MostRecTokenType}
    return compiled_ERC20_tnx_result

In [5]:
def get_address_erc20_tnx(data,address):
    sample_df = pd.DataFrame(data)

    # Column creation of ETH from Wei
    sample_df['eth value'] = sample_df['value'].apply(lambda x: Web3.fromWei(int(x),'ether'))

    # Typing of sent and received ERC20 transactions
    sample_df['txn type'] = sample_df['from'].apply(lambda x: 'sent' if x == address else 'received')
    sample_df['AccountId'] = address
    return sample_df

In [6]:
def get_empty_details_for_address(address):
    compiled_empty_address = {
          'Address': address, 'FLAG': 1,
          'Avg min between sent tnx': 0,
         'Avg min between received tnx': 0,
          'Time Diff between first and last (Mins)': 0,
          'Sent tnx': 0, 'Received Tnx': 0,
          'Number of Created Contracts': 0,
          'Unique Received From Addresses':0,
          'Unique Sent To Addresses': 0,
          'min value received': 0,
          'max value received ': 0,
          'avg val received': 0,
          'min val sent': 0,
          'max val sent': 0,
          'avg val sent': 0 ,
          'min value sent to contract': 0,
          'max val sent to contract': 0,
          'avg value sent to contract': 0,
          'total transactions (including tnx to create contract': 0,
          'total Ether sent': 0,
          'total ether received': 0,
          'total ether sent contracts': 0,
          'total ether balance':0,
        ' Total ERC20 tnxs': 0,
         ' ERC20 total Ether received': 0,
         ' ERC20 total ether sent': 0,
         ' ERC20 total Ether sent contract': 0,
         ' ERC20 uniq sent addr': 0,
         ' ERC20 uniq rec addr': 0,
         ' ERC20 uniq sent addr.1': 0 ,
         ' ERC20 uniq rec contract addr': 0,
         ' ERC20 avg time between sent tnx': 0,
         ' ERC20 avg time between rec tnx': 0,
         ' ERC20 avg time between rec 2 tnx': 0,
         ' ERC20 avg time between contract tnx': 0,
         ' ERC20 min val rec': 0,
         ' ERC20 max val rec': 0 ,
         ' ERC20 avg val rec': 0,
         ' ERC20 min val sent': 0,
         ' ERC20 max val sent': 0,
         ' ERC20 avg val sent': 0,
         ' ERC20 min val sent contract': 0,
         ' ERC20 max val sent contract': 0,
         ' ERC20 avg val sent contract': 0,
         ' ERC20 uniq sent token name': 0,
         ' ERC20 uniq rec token name': 0,
         ' ERC20 most sent token type': [[]],
         ' ERC20_most_rec_token_type': [[]]
    }
    return pd.DataFrame(compiled_empty_address)

In [7]:
def get_empty_normal_txn_details_for_address(address):
    compiled_empty_address = {
          'Address': address, 'FLAG': 1,
          'Avg min between sent tnx': 0,
         'Avg min between received tnx': 0,
          'Time Diff between first and last (Mins)': 0,
          'Sent tnx': 0, 'Received Tnx': 0,
          'Number of Created Contracts': 0,
          'Unique Received From Addresses':0,
          'Unique Sent To Addresses': 0,
          'min value received': 0,
          'max value received ': 0,
          'avg val received': 0,
          'min val sent': 0,
          'max val sent': 0,
          'avg val sent': 0 ,
          'min value sent to contract': 0,
          'max val sent to contract': 0,
          'avg value sent to contract': 0,
          'total transactions (including tnx to create contract': 0,
          'total Ether sent': 0,
          'total ether received': 0,
          'total ether sent contracts': 0,
          'total ether balance':0,
    }
    return compiled_empty_address

In [8]:
def get_empty_erc20_txn_details_for_address(address):
    compiled_empty_address = {
         'Address': address,
         ' Total ERC20 tnxs': 0,
         ' ERC20 total Ether received': 0,
         ' ERC20 total ether sent': 0,
         ' ERC20 total Ether sent contract': 0,
         ' ERC20 uniq sent addr': 0,
         ' ERC20 uniq rec addr': 0,
         ' ERC20 uniq sent addr.1': 0 ,
         ' ERC20 uniq rec contract addr': 0,
         ' ERC20 avg time between sent tnx': 0,
         ' ERC20 avg time between rec tnx': 0,
         ' ERC20 avg time between rec 2 tnx': 0,
         ' ERC20 avg time between contract tnx': 0,
         ' ERC20 min val rec': 0,
         ' ERC20 max val rec': 0 ,
         ' ERC20 avg val rec': 0,
         ' ERC20 min val sent': 0,
         ' ERC20 max val sent': 0,
         ' ERC20 avg val sent': 0,
         ' ERC20 min val sent contract': 0,
         ' ERC20 max val sent contract': 0,
         ' ERC20 avg val sent contract': 0,
         ' ERC20 uniq sent token name': 0,
         ' ERC20 uniq rec token name': 0,
         ' ERC20 most sent token type': [[]],
         ' ERC20_most_rec_token_type': [[]]
    }
    return pd.DataFrame(compiled_empty_address)

In [9]:
def get_empty_normal_txn_for_address(address):
    empty_txn = {'blockNumber':	np.nan,
                 'timeStamp': np.nan,
                 'hash': np.nan,
                 'nonce': np.nan,
                 'blockHash': np.nan,
                 'transactionIndex': np.nan,
                 'from': np.nan,
                 'to': np.nan,
                 'value': np.nan,
                 'gas': np.nan,
                 'gasPrice': np.nan,
                 'isError': np.nan,
                 'txreceipt_status': np.nan,
                 'input': np.nan,
                 'contractAddress': np.nan,
                 'cumulativeGasUsed': np.nan,
                 'gasUsed': np.nan,
                 'confirmations': np.nan,
                 'methodId': np.nan,
                 'functionName': np.nan,
                 'eth value': np.nan,
                 'txn type': np.nan,
                 'AccountId':address
}
    return pd.DataFrame(empty_txn, index=[0])

In [10]:
def get_empty_erc20_txn_for_address(address):
    empty_txn = {'blockNumber':	np.nan,
                 'timeStamp': np.nan,
                 'hash': np.nan,
                 'nonce': np.nan,
                 'blockHash': np.nan,
                 'from': np.nan,
                 'contractAddress': np.nan,
                 'to': np.nan,
                 'value': np.nan,
                 'tokenName': np.nan,
                 'tokenSymbol': np.nan,
                 'tokenDecimal': np.nan,
                 'transactionIndex': np.nan,
                 'gas': np.nan,
                 'gasPrice': np.nan,
                 'gasUsed': np.nan,
                 'cumulativeGasUsed': np.nan,
                 'input': np.nan,
                 'confirmations': np.nan,
                 'eth value': np.nan,
                 'txn type': np.nan,
                 'AccountId':address
}
    return pd.DataFrame(empty_txn, index=[0])

In [11]:
def get_details_for_address(normal_txn_data,erc20_txn_data,address):
    normal_address_stats = get_address_stats_normal_tnx(normal_txn_data,address)
    ERC20_stats= get_address_stats_erc20_tnx(erc20_txn_data,address)
    final_stats = {}
    final_stats.update(normal_address_stats)
    final_stats.update(ERC20_stats)
    return pd.DataFrame(final_stats)

In [12]:
# def get_final_details_for_address(normal_txn_data,erc20_txn_data,address):
#     try:
#         df = get_details_for_address(normal_txn_data,erc20_txn_data,address)
#     except:
#         df = get_empty_details_for_address(address)
#     return df

In [13]:
# def get_normal_txn_data(data,address):
#     try:
#         normal_txns = get_address_normal_tnx(data,address)
#     except:
#         normal_txns = get_empty_normal_txn_for_address(address)
#     return normal_txns

In [14]:
# def get_erc20_txn_data(data,address):
#     try:
#         erc20_txns = get_address_erc20_tnx(data,address)
#     except:
#         erc20_txns = get_empty_erc20_txn_for_address(address)
#     return erc20_txns

In [15]:
# def get_all_data(address):
#     try:
#         normal_txn_data = eth.get_normal_txs_by_address(address=address, startblock=0,endblock=99999999,sort='asc')
#         erc20_txn_data = eth.get_erc20_token_transfer_events_by_address(address=address,startblock=0,endblock=99999999,sort='asc')
#         data_manual_aggregate = get_details_for_address(normal_txn_data,erc20_txn_data,address)
#         processed_normal_txn_data = get_address_normal_tnx(normal_txn_data,address)
#         processed_erc20_txn_data = get_address_erc20_tnx(erc20_txn_data,address)
#     except:
#         data_manual_aggregate = get_empty_details_for_address(address)
#         processed_normal_txn_data = get_empty_normal_txn_for_address(address)
#         processed_erc20_txn_data = get_empty_erc20_txn_for_address(address)
#     return [data_manual_aggregate,processed_normal_txn_data,processed_erc20_txn_data]

In [16]:
def get_all_data(address):
    try:
        normal_txn_data = eth.get_normal_txs_by_address(address=address, startblock=0,endblock=99999999,sort='asc')
        processed_normal_txn_data = get_address_normal_tnx(normal_txn_data,address)
    except:
        processed_normal_txn_data = get_empty_normal_txn_for_address(address)

    try:
        erc20_txn_data = eth.get_erc20_token_transfer_events_by_address(address=address,startblock=0,endblock=99999999,sort='asc')
        processed_erc20_txn_data = get_address_erc20_tnx(erc20_txn_data,address)
    except:
        processed_erc20_txn_data = get_empty_erc20_txn_for_address(address)
    return [processed_normal_txn_data,processed_erc20_txn_data]

In [17]:
def get_new_feature_list(path,cand_path):
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
    base_df = pd.read_csv(cand_path + onlyfiles[0])
    base_df = base_df.drop(columns=['Unnamed: 0'])
    processing_count = 0
    num_of_files = len(onlyfiles)
    for i in onlyfiles:
        if i == onlyfiles[0]:
            continue
        else:
            temp_df = pd.read_csv(cand_path + i)
            temp_df  = temp_df .drop(columns=['Unnamed: 0'])
            base_df = pd.concat([base_df,temp_df])
        processing_count+=1
        print("{} out of {} combined".format(processing_count,num_of_files))
    base_df = base_df.dropna(axis=1,how='all')
    base_df['timeStamp']= base_df['timeStamp'].fillna(0)
    base_df['date time'] = base_df['timeStamp'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
    base_df['date time'] = base_df['date time'].apply(pd.to_datetime)
    base_df['eth value']= base_df['eth value'].fillna(0)
    base_df = base_df.drop(columns=['timeStamp'])
    base_df['adjusted_eth_value'] = np.where(base_df['txn type']=='sent',base_df['eth value']*-1,base_df['eth value'])
    final_df = base_df.dropna(axis=1,how='any')
    final_df = final_df.drop(columns='eth value')
    zero_txn_accounts = final_df.groupby('AccountId').filter(lambda x : len(x)<=1)
    non_zero_txn_accounts = final_df.groupby('AccountId').filter(lambda x : len(x)>1)
    df_features = extract_features(non_zero_txn_accounts,column_id='AccountId',
                                   column_sort='date time',column_value='adjusted_eth_value')
    df_features_indexed = df_features.reset_index()
    df_features_indexed = df_features_indexed.rename(columns={'index':"Address"})
    df_features_intermediate = zero_txn_accounts[['AccountId']]
    df_features_intermediate = df_features_intermediate.rename(columns={'AccountId':"Address"})
    df_final = pd.concat([df_features_indexed,df_features_intermediate])
    df_final = df_final.fillna(0).reset_index(drop=True)
    return df_final

# Execution on Mining and feature generation

In [48]:
ref_df = pd.read_csv('../Data/address_data_combined.csv')

In [49]:
ref_list = ref_df['Address'].values.tolist()

In [50]:
len(ref_list[7078])

42

In [51]:
ref_list = ref_list[7078:]

In [ ]:
counter = 7077
for i in ref_list:
    result = get_all_data(i)
    normal_txns = result[0]
    erc20_txns = result[1]
    normal_txn_fname = f'../Data/Transaction_data/Normal_txn/{i}_normal_txn.csv'
    erc20_txn_fname = f'../Data/Transaction_data/Erc20_txn/{i}_erc20_txn.csv'
    normal_txns.to_csv(normal_txn_fname)
    erc20_txns.to_csv(erc20_txn_fname)
    counter+=1
    print("{} out of 14155 mined".format(counter))

7078 out of 14155 mined
7079 out of 14155 mined
7080 out of 14155 mined
7081 out of 14155 mined
7082 out of 14155 mined
7083 out of 14155 mined
7084 out of 14155 mined
7085 out of 14155 mined
7086 out of 14155 mined
7087 out of 14155 mined


In [17]:
path = r'../Data/Transaction_data/Normal_txn'
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
cand_df = pd.read_csv('../Data/Transaction_data/Normal_txn/' + onlyfiles[0])
cand_df = cand_df.drop(columns=['Unnamed: 0'])
address = str(cand_df.iloc[1]['AccountId'])
base_df = pd.DataFrame(get_address_stats_normal_tnx(cand_df,address),index=[0])

In [18]:
counter = 0
for i in onlyfiles:
    if i == onlyfiles[0]:
        continue
    else:
        cand_df = pd.read_csv('../Data/Transaction_data/Normal_txn/' + i)
        cand_df = cand_df.drop(columns=['Unnamed: 0'])
        address = str(cand_df.iloc[0]['AccountId'])
        try:
            temp_df = pd.DataFrame(get_address_stats_normal_tnx(cand_df,address),index=[0])
        except:
            temp_df = pd.DataFrame(get_empty_normal_txn_details_for_address(address),index=[0])
        base_df = pd.concat([base_df,temp_df])
    counter+=1
    print("{} out of 13,920 mined".format(counter))
base_df = base_df.reset_index(drop=True)

1 out of 13,920 mined
2 out of 13,920 mined
3 out of 13,920 mined
4 out of 13,920 mined
5 out of 13,920 mined
6 out of 13,920 mined
7 out of 13,920 mined
8 out of 13,920 mined
9 out of 13,920 mined
10 out of 13,920 mined
11 out of 13,920 mined
12 out of 13,920 mined
13 out of 13,920 mined
14 out of 13,920 mined
15 out of 13,920 mined
16 out of 13,920 mined
17 out of 13,920 mined
18 out of 13,920 mined
19 out of 13,920 mined
20 out of 13,920 mined
21 out of 13,920 mined
22 out of 13,920 mined
23 out of 13,920 mined
24 out of 13,920 mined
25 out of 13,920 mined
26 out of 13,920 mined
27 out of 13,920 mined
28 out of 13,920 mined
29 out of 13,920 mined
30 out of 13,920 mined
31 out of 13,920 mined
32 out of 13,920 mined
33 out of 13,920 mined
34 out of 13,920 mined
35 out of 13,920 mined
36 out of 13,920 mined
37 out of 13,920 mined
38 out of 13,920 mined
39 out of 13,920 mined
40 out of 13,920 mined
41 out of 13,920 mined
42 out of 13,920 mined
43 out of 13,920 mined
44 out of 13,920 min

In [22]:
base_df.to_csv('../Data/Transaction_data/Normal_txn_aggregation.csv')

In [26]:
path = r'../Data/Transaction_data/Erc20_txn'
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
cand_df = pd.read_csv('../Data/Transaction_data/Erc20_txn/' + onlyfiles[0])
cand_df = cand_df.drop(columns=['Unnamed: 0'])
address = str(cand_df.iloc[0]['AccountId'])
try:
    base_df = pd.DataFrame(get_address_stats_erc20_tnx(cand_df,address),index=[0])
except:
    base_df = pd.DataFrame(get_empty_erc20_txn_details_for_address(address),index=[0])

In [27]:
counter = 0
for i in onlyfiles:
    if i == onlyfiles[0]:
        continue
    else:
        cand_df = pd.read_csv('../Data/Transaction_data/Erc20_txn/' + i)
        cand_df = cand_df.drop(columns=['Unnamed: 0'])
        address = str(cand_df.iloc[0]['AccountId'])
        try:
            temp_df = pd.DataFrame(get_address_stats_erc20_tnx(cand_df,address),index=[0])
        except:
            temp_df = pd.DataFrame(get_empty_erc20_txn_details_for_address(address),index=[0])
        base_df = pd.concat([base_df,temp_df])
    counter+=1
    print("{} out of 13,920 mined".format(counter))
base_df = base_df.reset_index(drop=True)

1 out of 13,920 mined
2 out of 13,920 mined
3 out of 13,920 mined
4 out of 13,920 mined
5 out of 13,920 mined
6 out of 13,920 mined
7 out of 13,920 mined
8 out of 13,920 mined
9 out of 13,920 mined
10 out of 13,920 mined
11 out of 13,920 mined
12 out of 13,920 mined
13 out of 13,920 mined
14 out of 13,920 mined
15 out of 13,920 mined
16 out of 13,920 mined
17 out of 13,920 mined
18 out of 13,920 mined
19 out of 13,920 mined
20 out of 13,920 mined
21 out of 13,920 mined
22 out of 13,920 mined
23 out of 13,920 mined
24 out of 13,920 mined
25 out of 13,920 mined
26 out of 13,920 mined
27 out of 13,920 mined
28 out of 13,920 mined
29 out of 13,920 mined
30 out of 13,920 mined
31 out of 13,920 mined
32 out of 13,920 mined
33 out of 13,920 mined
34 out of 13,920 mined
35 out of 13,920 mined
36 out of 13,920 mined
37 out of 13,920 mined
38 out of 13,920 mined
39 out of 13,920 mined
40 out of 13,920 mined
41 out of 13,920 mined
42 out of 13,920 mined
43 out of 13,920 mined
44 out of 13,920 min

In [ ]:
base_df.to_csv('../Data/Transaction_data/Erc20_txn_aggregation.csv')

In [19]:
path = r'../Data/Transaction_data/Normal_txn'
sub_path = '../Data/Transaction_data/Normal_txn/'
normal_txn_new_features = get_new_feature_list(path,sub_path)

1 out of 13920 combined
2 out of 13920 combined
3 out of 13920 combined
4 out of 13920 combined
5 out of 13920 combined
6 out of 13920 combined
7 out of 13920 combined
8 out of 13920 combined
9 out of 13920 combined
10 out of 13920 combined
11 out of 13920 combined
12 out of 13920 combined
13 out of 13920 combined
14 out of 13920 combined
15 out of 13920 combined
16 out of 13920 combined
17 out of 13920 combined
18 out of 13920 combined
19 out of 13920 combined
20 out of 13920 combined
21 out of 13920 combined
22 out of 13920 combined
23 out of 13920 combined
24 out of 13920 combined
25 out of 13920 combined
26 out of 13920 combined
27 out of 13920 combined
28 out of 13920 combined
29 out of 13920 combined
30 out of 13920 combined
31 out of 13920 combined
32 out of 13920 combined
33 out of 13920 combined
34 out of 13920 combined
35 out of 13920 combined
36 out of 13920 combined
37 out of 13920 combined
38 out of 13920 combined
39 out of 13920 combined
40 out of 13920 combined
41 out of

Feature Extraction: 100%|██████████| 30/30 [1:33:30<00:00, 187.01s/it]


In [21]:
normal_txn_new_features.to_csv('../Data/Transaction_data/normal_txn_TS_features.csv')

# Getting new Time Series Dataset

In [105]:
normal_txn_new_features = pd.read_csv('../Data/Transaction_data/normal_txn_TS_features.csv')
label_data = pd.read_csv('../Data/address_data_combined.csv')

In [107]:
normal_txn_new_features = normal_txn_new_features.drop(columns=['Unnamed: 0'])

In [110]:
label_data = label_data[['Address','FLAG']]

In [112]:
compiled_df_only_new = pd.merge(normal_txn_new_features,label_data,on='Address',how='left')

In [115]:
# Removing Flag Label
feature_transaction_df = compiled_df_only_new.drop(columns = 'FLAG')

# Plotting Correlation Matrix to see understand which features are highly correlated
numeric_transaction_df = feature_transaction_df = compiled_df_only_new.drop(columns = 'FLAG').select_dtypes(include=np.number) #
corr_matrix = numeric_transaction_df.corr().round(2)
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

In [116]:
correlation_threshold = 0.7
reduced_corr_matrix = corr_matrix.mask(mask)
corr_matrix_unstacked = reduced_corr_matrix.unstack()
similar_features = corr_matrix_unstacked[abs(corr_matrix_unstacked) >= correlation_threshold].reset_index().rename(columns={'level_0': 'feature_1', 'level_1': 'feature_2', 0: 'Absolute Correlation'})
similar_features.sort_values(by="Absolute Correlation",ascending=False)

,feature_1,feature_2,Absolute Correlation
848,adjusted_eth_value__symmetry_looking__r_0.5,adjusted_eth_value__symmetry_looking__r_0.65,1.00
3613,"adjusted_eth_value__change_quantiles__f_agg_""v...","adjusted_eth_value__change_quantiles__f_agg_""v...",1.00
1055,adjusted_eth_value__number_peaks__n_3,adjusted_eth_value__number_peaks__n_5,1.00
946,adjusted_eth_value__symmetry_looking__r_0.9500...,adjusted_eth_value__autocorrelation__lag_0,1.00
804,adjusted_eth_value__symmetry_looking__r_0.3500...,adjusted_eth_value__symmetry_looking__r_0.8,1.00
...,...,...,...
1132,adjusted_eth_value__cwt_coefficients__coeff_0_...,adjusted_eth_value__cwt_coefficients__coeff_10...,-0.98
1396,adjusted_eth_value__cwt_coefficients__coeff_1_...,adjusted_eth_value__cwt_coefficients__coeff_11...,-0.98
2387,adjusted_eth_value__cwt_coefficients__coeff_5_...,adjusted_eth_value__cwt_coefficients__coeff_14...,-0.98
1955,adjusted_eth_value__cwt_coefficients__coeff_3_...,adjusted_eth_value__cwt_coefficients__coeff_12...,-0.99


In [117]:
feature_to_remove_set = similar_features['feature_2'].unique()

In [118]:
feature_to_remove_set = feature_to_remove_set.tolist()

In [119]:
compiled_df_only_new = compiled_df_only_new.drop(columns=feature_to_remove_set,axis=1)

In [120]:
compiled_df_only_new

,Address,adjusted_eth_value__variance_larger_than_standard_deviation,adjusted_eth_value__has_duplicate_max,adjusted_eth_value__has_duplicate_min,adjusted_eth_value__has_duplicate,adjusted_eth_value__sum_values,adjusted_eth_value__abs_energy,adjusted_eth_value__mean_abs_change,adjusted_eth_value__mean_second_derivative_central,adjusted_eth_value__median,...,adjusted_eth_value__ratio_beyond_r_sigma__r_3,adjusted_eth_value__ratio_beyond_r_sigma__r_5,adjusted_eth_value__ratio_beyond_r_sigma__r_6,adjusted_eth_value__ratio_beyond_r_sigma__r_10,adjusted_eth_value__count_above__t_0,adjusted_eth_value__count_below__t_0,adjusted_eth_value__fourier_entropy__bins_2,adjusted_eth_value__query_similarity_count__query_None__threshold_0.0,"adjusted_eth_value__matrix_profile__feature_""min""__threshold_0.98",FLAG
0,0x000000000532b45f47779fce440748893b257865,1.0,0.0,1.0,1.0,0.002552,5587.558824,5.789218,-0.237797,5.000000,...,0.000000,0.000000,0.000000,0.0,0.869565,0.130435,0.450561,0.0,0.000000,1
1,0x00000000072d54638c2c2a3da3f715360269eea1,1.0,0.0,0.0,1.0,-0.771623,52949.011552,10.615707,2.037385,0.033404,...,0.018868,0.018868,0.018868,0.0,0.811321,0.433962,0.479166,0.0,0.634623,1
2,0x0000000086c5d614bec59dfd2c9b88f7cb57f23c,1.0,0.0,0.0,0.0,-9.999391,4444.468528,20.771913,-8.228324,5.162142,...,0.000000,0.000000,0.000000,0.0,0.833333,0.166667,0.562335,0.0,0.000000,1
3,0x00000000d029a3ed50d891b9afd74b034179082e,0.0,0.0,0.0,0.0,0.004158,0.495859,0.995842,0.000000,0.002079,...,0.000000,0.000000,0.000000,0.0,0.500000,0.500000,-0.000000,0.0,0.000000,1
4,0x00000e32e51011e28958d4696627c82c3dacd5a6,0.0,0.0,0.0,1.0,-0.035777,0.351082,0.098646,-0.000387,0.020000,...,0.023256,0.000000,0.000000,0.0,0.720930,0.302326,0.625491,0.0,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13915,0xfea4384f85fe123f87f393946118e3828187c28a,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,1
13916,0xff0e20e3a7575d823302bbb27e233c23f2c3070b,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,1
13917,0xff4a9b2da613b7013767d91af302acf7c06e4eed,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,1
13918,0xff718805bb9199ebf024ab6acd333e603ad77c85,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,1


In [121]:
compiled_df_only_new.to_csv('../Data/new_ts_features_only.csv')